In [5]:
import json
import pandas as pd

from detectron2.data import (
    MetadataCatalog,
    DatasetCatalog,
    DatasetFromList,
    build_detection_test_loader,
    build_detection_train_loader,
)



In [11]:
jf = r"/media/ps/train/LQ/trt/trt/model/summary_sensitive_analysis.json"

jd = json.load(open(jf))
njd = {}
for info in jd:
    map = {}
    for key, value in info[0].items():
        for ap, v in value.items():
            map[f"{key}_{ap}"] = v
    njd[info[1]] = map
    

jpd = pd.DataFrame(njd).T
jpd.to_csv('/media/ps/train/LQ/trt/trt/model/map.csv')
    

In [ ]:
def infer(model, data_loader, num_batch=200):
    model.eval()
    quantize.disable_quantization(model)
    with torch.no_grad():
            for i, datas in tqdm(enumerate(data_loader), total=min(len(data_loader), num_batch), desc="Collect stats for calibrating"):
                inf_t0 = time_synchronized()
                model(datas)
                inf_t1 = time_synchronized()
                print("当前耗时推理耗时:", inf_t1 - inf_t0)
                if i >= num_batch:
                    break

In [ ]:
def setup(args):
    # train.py -----------------------------------------------------------------------------------------
    cfg = get_cfg()
    cfg.merge_from_list(args.opts)
    config_file = '/home/ps/adet/AdelaiDet/configs/BlendMask/R_50_3x.yaml'
    cfg.merge_from_file(config_file)

    cfg.DATASETS.TRAIN = ("phone_train",)
    cfg.DATASETS.TEST = ("phone_test",)   # no metrics implemented for this dataset
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.1
    cfg.MODEL.FCOS.INFERENCE_TH_TEST = 0.09

    cfg.MODEL.WEIGHTS = args.weights
    cfg.MODEL.DEVICE = "cuda:2"
    cfg.MODEL.ROI_HEADS.NUM_CLASSES = 50  # 3 classes (data, fig, hazelnut)
    cfg.MODEL.FCOS.NUM_CLASSES = 50

    cfg.MODEL.RESNETS.DEPTH = 34
    cfg.MODEL.RESNETS.RES2_OUT_CHANNELS = 64
    cfg.MODEL.BACKBONE.FREEZE_AT = 0
    cfg.MODEL.FCOS.CENTER_SAMPLE = "center"
        
    if args.channel == 1:
        cfg.INPUT.FORMAT = 'L'
        cfg.MODEL.PIXEL_MEAN = [90]
        cfg.MODEL.PIXEL_STD = [77]
    else:
        cfg.INPUT.FORMAT = 'BGR'
        cfg.MODEL.PIXEL_MEAN = [1,1,1]
        cfg.MODEL.PIXEL_STD = [1,1,1]

    cfg.MODEL.BASIS_MODULE.NORM = 'BN'
    cfg.freeze()
    return cfg

In [ ]:
def main():
    
    #数据集准备
    
    train_dataset_name = "phone_train"
    val_dataset_name = "phone_val"
    jf_val = '/media/ps/train/LQ/trt/trt/data/data2/val/train.json'
    imgs_val = '/media/ps/train/LQ/trt/trt/data/data2/val'
    register_coco_instances(val_dataset_name, {}, jf_val, imgs_val)

    jf_train = '/media/ps/train/LQ/trt/trt/data/data2/train/train.json'
    imgs_train = '/media/ps/train/LQ/trt/trt/data/data2/train'
    register_coco_instances(train_dataset_name, {}, jf_train, imgs_train)

    train_dataloader = build_detection_test_loader(cfg, train_dataset_name)
    val_dataloader   = build_detection_test_loader(cfg, val_dataset_name)
    parser = argparse.ArgumentParser(description="Export model to the onnx format")
    parser.add_argument(
        "--config-file",
        default="/home/ps/adet/AdelaiDet/configs/BlendMask/R_50_3x.yaml",
        metavar="FILE",
        help="path to config file",
    )
    parser.add_argument('--width', default=2048, type=int)
    parser.add_argument('--height', default=2048, type=int)
    parser.add_argument('--channel', default=1, type=int)
    parser.add_argument('--dynamic', default=False, action="store_true")
    parser.add_argument('--ptq', default=True, action="store_true")
    
    parser.add_argument(
        "--weights",
        default="/media/ps/train/LQ/trt/trt/model/model_0413999.pth",
        metavar="FILE",
        help="path to the output onnx file",
    )
    parser.add_argument(
        "--output",
        default="/media/ps/train/LQ/trt/trt/model",
        metavar="FILE",
        help="path to the output onnx file",
    )
    parser.add_argument(
        "--opts",
        help="Modify config options using the command-line 'KEY VALUE' pairs",
        default=[],
        nargs=argparse.REMAINDER,
    )

    args = parser.parse_args()
    cfg = setup(args)

    model = build_model(cfg)


    checkpointer = DetectionCheckpointer(model)
    _ = checkpointer.load(cfg.MODEL.WEIGHTS)    
    model.to(cfg.MODEL.DEVICE)
    model.eval()
    